<a href="https://colab.research.google.com/github/its-Ravi-Singh/Prac-NN-from-Scratch/blob/main/Pytorch_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1

In [19]:
import os
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

### Introduction to PyTorch Tensors

In [5]:
print("torch version:", torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

torch version: 2.9.0+cpu
device: cpu


In [6]:
# basic tensors
a = torch.tensor([3, 7, 11], dtype=torch.int64)
b = torch.tensor([1.5, 2.0, 3.25], dtype=torch.float32)

print(a, a.dtype)
print(b, b.dtype)

# create tensors quickly for some standad forms
z = torch.zeros((2, 4))
o = torch.ones((3, 1))
r = torch.rand((2, 3))
n = torch.randn((2, 3))

print("zeros:\n", z)
print("ones:\n", o)
print("rand:\n", r)
print("randn:\n", n)

# reshape them + indexing
x = torch.arange(0, 20)
x2 = x.reshape(4, 5)
print("x2 :\n", x2)
print("row_0:", x2[0, :])
print("col_2:", x2[:, 2])

# broadcasting
col = torch.tensor([[1.0], [2.0], [3.0]])
row = torch.tensor([10.0, 20.0, 30.0])
print("broadcast:\n", col + row)

tensor([ 3,  7, 11]) torch.int64
tensor([1.5000, 2.0000, 3.2500]) torch.float32
zeros:
 tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])
ones:
 tensor([[1.],
        [1.],
        [1.]])
rand:
 tensor([[0.0171, 0.1058, 0.7755],
        [0.4235, 0.9039, 0.4048]])
randn:
 tensor([[ 0.0062, -1.3743,  0.2986],
        [-0.5272, -2.0863, -1.1311]])
x2 :
 tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]])
row_0: tensor([0, 1, 2, 3, 4])
col_2: tensor([ 2,  7, 12, 17])
broadcast:
 tensor([[11., 21., 31.],
        [12., 22., 32.],
        [13., 23., 33.]])


### The Fundamentals of Autograd

In [7]:
w = torch.tensor(2.0, requires_grad=True) # autograd demo
y = w*w + 3*w + 1
y.backward()

print("y =", y.item())
print("dy/dw =", w.grad.item())

y = 11.0
dy/dw = 7.0


### Building Models with PyTorch

In [8]:
class RegNetwork(nn.Module):
    def __init__(self):
      super().__init__()
      # input has 2 features, output is 16 neurons
      self.fc1 = nn.Linear(2, 16)
      # takes 16 inputs and gives single output
      self.fc2 = nn.Linear(16, 1)

    def forward(self, x):
      x = self.fc1(x) # pass input through first layer
      x = torch.relu(x) # Apply Relu activation fn
      x = self.fc2(x) #pass through 2nd layer
      return x

model = RegNetwork().to(device)
print(model)

RegNetwork(
  (fc1): Linear(in_features=2, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=1, bias=True)
)


In [10]:
# regression data
torch.manual_seed(7)

X = torch.randn(800, 2)
y_true = 2.5 * X[:, 0] - 1.2 * X[:, 1] + 0.3
y = y_true + 0.15 * torch.randn(800)


# split data
perm = torch.randperm(X.shape[0])

train_ids = perm[:600]
val_ids   = perm[600:700]
test_ids  = perm[700:]

X_train, y_train = X[train_ids].to(device), y[train_ids].to(device)
X_val, y_val     = X[val_ids].to(device),   y[val_ids].to(device)
X_test, y_test   = X[test_ids].to(device),  y[test_ids].to(device)

print("train:", X_train.shape, y_train.shape)
print("val  :", X_val.shape, y_val.shape)
print("test :", X_test.shape, y_test.shape)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# TensorBoard
writer = SummaryWriter("runs/a0_part1_tinyreg_student")

# Train loop (saving best model using val loss)
best_val_loss = float("inf")
best_model_path = "a0_part_1_weights_best.pt"

t0 = time.time()

num_epochs = 100
for epoch in range(num_epochs):
    # training
    model.train()
    optimizer.zero_grad()

    y_pred = model(X_train).squeeze()       # (600,)
    train_loss = criterion(y_pred, y_train)

    train_loss.backward()
    optimizer.step()

    # -validation
    model.eval()
    with torch.no_grad():
        val_pred = model(X_val).squeeze()
        val_loss = criterion(val_pred, y_val)

    # log losses
    writer.add_scalar("loss/train", train_loss.item(), epoch + 1)
    writer.add_scalar("loss/val", val_loss.item(), epoch + 1)

    # save best
    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        torch.save(model.state_dict(), best_model_path)

    # print sometimes
    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:03d} | train loss: {train_loss.item():.4f} | val loss: {val_loss.item():.4f}")

print("training time (sec):", round(time.time() - t0, 2))

# 6) Test using best saved weights
model.load_state_dict(torch.load(best_model_path, map_location=device))
model.eval()

with torch.no_grad():
    test_pred = model(X_test).squeeze()
    test_loss = criterion(test_pred, y_test)

print("best val loss:", round(best_val_loss, 4))
print("test loss:", round(test_loss.item(), 4))

writer.close()


train: torch.Size([600, 2]) torch.Size([600])
val  : torch.Size([100, 2]) torch.Size([100])
test : torch.Size([100, 2]) torch.Size([100])
Epoch 020 | train loss: 0.0246 | val loss: 0.0264
Epoch 040 | train loss: 0.0233 | val loss: 0.0255
Epoch 060 | train loss: 0.0228 | val loss: 0.0254
Epoch 080 | train loss: 0.0226 | val loss: 0.0256
Epoch 100 | train loss: 0.0225 | val loss: 0.0257
training time (sec): 0.22
best val loss: 0.0251
test loss: 0.0232


# Part 2

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [21]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

df = pd.read_csv("Crime_Incidents_20260205.csv")

device: cpu


/tmp/ipython-input-2366659109.py:8: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Crime_Incidents_20260205.csv")


In [18]:
df.columns

Index(['Case Number', 'Incident Datetime', 'Incident ID',
       'Incident Type Primary', 'Incident Description', 'Parent Incident Type',
       'Hour of Day', 'Day of Week', 'Address', 'City', 'State', 'Location',
       'Latitude', 'Longitude', 'Created At', 'updated_at', 'zip_code',
       'neighborhood', 'Council District', 'Council District 2011',
       'Census Tract', 'Census Block Group', 'Census Block',
       '2010 Census Tract ', '2010 Census Block Group', '2010 Census Block',
       'Police District', 'TRACTCE20', 'GEOID20_tract', 'GEOID20_blockgroup',
       'GEOID20_block'],
      dtype='object')